In [ ]:
%load_ext autoreload
%autoreload 2

### Test wrapper on moons

Test that we can use `flowtorch` with transformations from `nflows`. 

Example is modifies  from [nflows](https://github.com/bayesiains/nflows)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
from tqdm import tqdm
import os
from itertools import chain
import seaborn as sns
import torch

import flowtorch.distributions as ftdist

from nflows.flows.base import Flow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.flows import realnvp, autoregressive

from animation import fetch_animation_id_set, fetch_animations
from so3.curves import move_origin_to_zero as move_origin_to_zero_so3
from so3.transformations import hatinv
from linear import animation_to_SO3
from linear.curves import move_origin_to_zero

from deepthermal.validation import (
    create_subdictionary_iterator,
    k_fold_cv_grid,
    add_dictionary_iterators,
)
from deepthermal.plotting import plot_result

from shapeflow import ModuleBijector, WrapInverseModel

# make reproducible
seed = torch.manual_seed(0)

# better formats
set_matplotlib_formats("pdf", "svg")

In [ ]:
# len(fetch_animation_id_set(description="walk"))

# # fetch data as so3
# print("loading mocap data:")
# animation_tuples = fetch_animations(1000, description="walk")

In [ ]:
# # reshape data
# skel, anim, desk = animation_tuples[0]
# # root elem
# num_bones = len(skel.bones) + 1
# num_anims =  len(animation_tuples)
# total_frames = sum(map(lambda t : len(t[1].get_frames()), animation_tuples))

# for s, a, d in animation_tuples:
#     assert len(s.bones) +1 == num_bones , f"not {num_bones} bones"

# anim_array = torch.cat([torch.as_tensor(animation_to_SO3(*t[:2])) for t in  tqdm(animation_tuples)], 1)
# frames = torch.moveaxis(anim_array,0,1)

In [ ]:
# save data since it takes so long to get
# torch.save(frames, 'walk_frames.pt')
frames = torch.load("walk_frames.pt")

In [ ]:
data = torch.utils.data.TensorDataset(torch.flatten(frames, 1))

In [ ]:
# Set up model
num_layers = 5
event_shape = 23 * 3 * 3
base_dist = torch.distributions.Independent(
    torch.distributions.Normal(torch.zeros(event_shape), torch.ones(event_shape)), 1
)

transforms = []
for _ in range(num_layers):
    transforms.append(ReversePermutation(features=event_shape))
    transforms.append(
        MaskedAffineAutoregressiveTransform(features=event_shape, hidden_features=4)
    )
transform = CompositeTransform(transforms)
bijector = WrapInverseModel(model=transform)

flow = ftdist.Flow(bijector=bijector, base_dist=base_dist)
optimizer = optim.Adam(flow.parameters())

In [ ]:
#######
DIR = "../figures/curve_1/"
SET_NAME = "walk_1"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########


FOLDS = 1
N = 128  # training points internal
lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode="min", factor=0.2, patience=5, verbose=True
)
loss_func = get_elastic_metric_loss(r=c1.r, constrain_cost=1e3, verbose=False)
no_penalty_loss_func = get_elastic_metric_loss(r=c1.r, constrain_cost=0, verbose=False)

MODEL_PARAMS = {
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": ["tanh"],
    "n_hidden_layers": [1, 2, 8, 32, 128],
    "neurons": [8],
}
MODEL_PARAMS_2 = {
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": ["tanh"],
    "n_hidden_layers": [2],
    "neurons": [32],
}

# extend the previous dict with the zip of this
MODEL_PARAMS_EXPERIMENT = {
    "model": [BResCNN, ResNet],
    "neurons": [8, 8],
    "kernel_size": [7, 0],
}
TRAINING_PARAMS = {
    "batch_size": [N // 2],
    "regularization_param": [0],
    "loss_func": [loss_func],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "optimizer": ["strong_wolfe"],
    "num_epochs": [100],
    "learning_rate": [1],
    "lr_scheduler": [lr_scheduler],
    "track_epoch": [True],
}

Test that the wrapper works

In [ ]:
num_iter = 5000
for i in range(num_iter):

    optimizer.zero_grad()
    loss = -flow.log_prob(value=data).mean()

    loss.backward()
    optimizer.step()

In [ ]:
sample = flow.sample([100]).data
print(sample)